'\nThis script writes a fasta with the infered ancestral sequence.\nNote that ancestral state is only infered for SNP sites.\nHence, the resulting ancestral-fasta should only be used \nin the context of the same\nSNP calls that the ancestral-fasta was produced with.\n'

In [1]:
#parallel
from ipyparallel import Client
rc = Client()
lv = rc.load_balanced_view()

In [2]:
%%px --local
import vcf as vcftools
import pyfasta, os
eu = os.path.expanduser
jn = os.path.join

In [3]:
chromosomes = ["CAE" + str(i) for i in range(1,30)+["X","Y"]]

In [4]:
%%px --local
#ana_dir = eu("~/vervet_project/analyses/20140403_UnifiedGenotyper_ref3500_non_VRC/_data/")
#vcf_fn = jn(ana_dir,"GATK_UnifiedGenotyper_163NonVRC_ref3500_snp_{}_including_filtered.vcf")
#ref_fn = eu("~/vervet_project/data/reference/reference_3500/"
#            "3500_indID1_codeVRC_ref_sequencer3_seqType1_filtered0_version6.fasta")
#ref_seq = pyfasta.Fasta(ref_fn)
ana_dir = eu("~/vervet_project/analyses/20150504_163_UnifiedGenotyper/_data/")
vcf_fn = jn(ana_dir,"163_UG_ref3500_{}_snp_pass.vcf.gz")
ref_fn = eu("~/vervet_project/data/reference/reference_3500/"
            "3500_indID1_codeVRC_ref_sequencer3_seqType1_filtered0_version6.fasta")
ref_seq = pyfasta.Fasta(ref_fn)

## Write Ancestral fasta with only info at SNP sites (read below!!!)

In [ ]:
"""
This script writes a fasta with the infered ancestral sequence.
Note that ancestral state is only infered for SNP sites.
Hence, the resulting ancestral-fasta should only be used 
in the context of the same
SNP calls that the ancestral-fasta was produced with.
"""

In [34]:
%%px --local
def get_ancestral_seq(vcf_fn,ref_seq):
    """
    Make a sequence string starting with the reference
    where at each snp the allele is subsituted by the
    ancestral allele (if known).
    """
    anc_state_count =0
    seq_ls = []
    prev_pos = 0
    vcf = vcftools.Reader(filename=vcf_fn)
    for i,record in enumerate(vcf):
        pos = record.POS-1
        seq_ls.append(ref_seq[record.CHROM][prev_pos:pos])
        try:
            anc = record.INFO['AA']
            if anc == record.REF or anc == record.ALT[0]:
                allele = anc
                anc_state_count += 1
            else:
                allele = record.REF
        except KeyError:
            allele = record.REF
        seq_ls.append(allele)
        prev_pos = pos + 1
        #if i > 1000:
        #    break
    seq_ls.append(ref_seq[record.CHROM][prev_pos:])
    print i,"snps processed"
    print anc_state_count,"have ancestral state info that is ref or alt"
    return "".join(seq_ls)

In [35]:
%%px --local
def write_to_fasta(handle,chrom,seq_str,line_len=60):
    handle.write(">{}\n".format(chrom))
    for idx in range(line_len,len(seq_str),line_len):
        handle.write(seq_str[idx-line_len:idx]+"\n")
    if seq_str[idx:]:
        handle.write(seq_str[idx:]+"\n")

In [15]:
#test
chrom = "CAE1"
anc_seq = get_ancestral_seq(vcf_fn.format(chrom),ref_seq)
with open("test_seq.fa","w") as fa:
    write_to_fasta(fa,chrom,anc_seq)

KeyboardInterrupt: 

In [36]:
%%px --local
out_fn = jn(ana_dir,"UG_163_vs_ref3500_ancestral_seq_{}.fa")
def write_anc_fasta(chrom):
    anc_seq = get_ancestral_seq(vcf_fn.format(chrom),ref_seq)
    with open(out_fn.format(chrom),"w") as fa:
        write_to_fasta(fa,chrom,anc_seq)

In [37]:
chrom_fn_str = " ".join([jn(ana_dir,"UG_163_vs_ref3500_ancestral_seq_{}.fa".format(chrom)) for chrom in chromosomes])

In [21]:
anc_seqs

<AsyncMapResult: <lambda>>

In [18]:
anc_seqs = dview.map_async(lambda chrom: write_anc_fasta(chrom), chromosomes)

In [107]:
%%bash -s "$chrom_fn_str" "$ana_dir"
#join chromosome fasta files
cd $2
rm UG_163_vs_ref3500_ancestral_seq2.fa
cat $1 >> UG_163_vs_ref3500_ancestral_seq2.fa 
#add scaffolds from original vcf
tail -n +45735302 ~/vervet_project/data/reference/reference_3500/3500_indID1_codeVRC_ref_sequencer3_seqType1_filtered0_version6.fasta >>  UG_163_vs_ref3500_ancestral_seq2.fa 

### second possibility, write line by line instead of reading everything in memory

In [5]:
%%px --local
def write_ancestral_seq(vcf_fn, chrom,ref_seq, out_file, line_len):
    """
    Write sequence fasta where for each snp the ancestral state is given.
    Write N for all other sites.
    """
    def line_write(handle,line_pos,string):

        idx_last = 0
        intervals = range(line_len-line_pos,len(string),line_len) + [len(string)]
        
        for i,idx in enumerate(intervals):
            slice0 = string[idx_last:idx]
            line_pos = line_pos + len(slice0)
            handle.write(slice0)
            if line_pos == line_len:
                handle.write("\n")
                line_pos = 0
            idx_last=idx
        return line_pos
    
    try:
        out = open(out_file,"w") #if filename
    except TypeError:
        out = out_file #if file handle
    line_pos = 0 # 1-based index
    anc_state_count =0
    prev_pos = 0
    
    out.write(">{}\n".format(chrom))
    vcf = vcftools.Reader(filename=vcf_fn)
    for i,record in enumerate(vcf):
        pos = record.POS-1
        line_pos = line_write(out,line_pos,"N"*(pos-prev_pos))
        try:
            anc = record.INFO['AA']
            allele = anc
            anc_state_count += 1
        except KeyError:
            allele = "N"
        line_pos = line_write(out,line_pos,allele)
        prev_pos = pos + 1
    line_pos = line_write(out,line_pos,
                          "N"*(len(ref_seq[record.CHROM])-prev_pos))
    if line_pos != 0:
        out.write("\n")
    print i,"snps processed"
    print anc_state_count,"have ancestral state info that is ref or alt"
    return (i,anc_state_count)

In [23]:
%%px --local
anc_state_N_fn = jn(ana_dir,"UG_163_ref3500_anc_state_snp_rest_N_{}.fa")

In [24]:
anc_state_N_fns = " ".join(anc_state_N_fn.format(chrom) for chrom in chromosomes[:-1])

In [8]:
anc_seqs = lv.map_async(lambda chrom:write_ancestral_seq(vcf_fn.format(chrom), 
                                                            chrom,ref_seq,
                                                            anc_state_N_fn.format(chrom),
                                                            line_len=60), chromosomes)

In [10]:
anc_seqs.wait(10)

In [12]:
anc_seqs.result

CompositeError: one or more exceptions from call to method: <lambda>
[0:apply]: NameError: global name 'handle' is not defined
[4:apply]: NameError: global name 'handle' is not defined
[10:apply]: NameError: global name 'handle' is not defined
[8:apply]: NameError: global name 'handle' is not defined
.... 27 more exceptions ...

In [ ]:
for r in anc_seqs:
    print r

In [74]:
anc_seqs.wait(5)

In [92]:
%%bash
echo X && echo Y

X
Y


In [26]:
%%bash -s "$anc_state_N_fns" "$ana_dir"
#join chromosome fasta files
cd $2
rm UG_163_ref3500_anc_state_snp_rest_N.fa 
for fn in `echo $1`; do (cat $fn && echo) >> UG_163_ref3500_anc_state_snp_rest_N.fa; done
#rm UG_163_vs_ref3500_ancestral_seq2.fa
#cat $1 > UG_163_ref3500_anc_state_snp_rest_N.fa 
#add scaffolds from original vcf
#tail -n +45735302 ~/vervet_project/data/reference/reference_3500/3500_indID1_codeVRC_ref_sequencer3_seqType1_filtered0_version6.fasta >>  UG_163_ref3500_anc_state_snp_rest_N.fa
#add Y and scaffolds
tail -n +45632280 ~/vervet_project/data/reference/reference_3500/3500_indID1_codeVRC_ref_sequencer3_seqType1_filtered0_version6.fasta >>  UG_163_ref3500_anc_state_snp_rest_N.fa 

In [192]:
!cat test456.fa

>CAE1
GTGAAAGAAGCCAAAAAGGCTACATACTGTATGATTCCAATTACGTGACATTTTGGAAAA
GGGAAAATTACAAAGATAGTAAAACAATCAATAGTTGACAGGGGTTGGGGAGATTGGGGA
GAGGAAAGGATGGATGGGCAGAGAGCCCCGGCTGTTTAGGGCAGTGAGACCACTGTGGGT
GATTTCACCACAACGGTGGACCCACGACAGCGTGCAGATGCCCACACCCATTGAATTAA

In [173]:
teststrings = ["kurz","K",
               "EinSEHRSEHRSEHRLAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA"
               "AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAANGERstring","USW","USF"]

In [44]:
def line_write(handle,line_pos,string,line_len=60):
        
    idx_last = 0
    intervals = range(line_len-line_pos,len(string),line_len) + [len(string)] 
    #print intervals
    for i,idx in enumerate(intervals):
        slice0 = string[idx_last:idx]
        line_pos = line_pos + len(slice0)
        handle.write(slice0)
        if line_pos == line_len:
            handle.write("\n")
            line_pos = 0
        idx_last=idx
    return line_pos

In [189]:
line_pos = 0
for string in teststrings:
    line_pos = line_write(sys.stdout,line_pos,string)

kurzKEinSEHRSEHRSEHRLAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAANGERstringUSWUSF

## Write ancestral fasta (macaque alignment in vervet coordinates)

In [196]:
#%%px --local
from hs_vervet.scripts import nucmer_ancestral_state_to_vcf as nas
import pyfasta
nucmer_dir = eu("~/vervet_project/analyses/20140319_nucmer_align_ref3500_vs_macaque/_data/")
mac_fasta = pyfasta.Fasta(eu("~/vervet_project/data/other_species/Macaca_mulatta/CR_1.0/Primary_Assembly/"
                "assembled_chromosomes/FASTA/Macaca_mulatta_CR_1.0.fasta"))
ver_fasta = pyfasta.Fasta(eu("~/vervet_project/data/reference/reference_3500/"
                             "3500_indID1_codeVRC_ref_sequencer3_seqType1_filtered0_version6.fasta"))


KeyboardInterrupt: 

In [49]:
def get_mac_fasta_in_ver_coord(chrom):
    delta_fn = jn(nucmer_dir,"nucmer_macaque_align_{}_filter_1_l200_u80.delta".format(chrom))
    query_ref_dic = nas.parse_delta_file(delta_fn,mac_fasta)
    positions = query_ref_dic.keys()
    mac_fasta_in_ver_coord = []
    for i in range(len(ver_fasta[chrom])):
        try:
            mac_fasta_in_ver_coord.append(query_ref_dic[i])
        except KeyError:
            mac_fasta_in_ver_coord.append('N')
    mac_fasta_in_ver_coord = "".join(mac_fasta_in_ver_coord)
    return mac_fasta_in_ver_coord

In [50]:
map_fasta = lv.map_async(get_mac_fasta_in_ver_coord,chromosomes[:-1])

In [68]:
chrom = "CAE28"

In [70]:
delta_fn

'/home/GMI/hannes.svardal/vervet_project/analyses/20140319_nucmer_align_ref3500_vs_macaque/_data/nucmer_macaque_align_CAE28_filter_1_l200_u80.delta'

In [75]:
import math

In [80]:
len(ver_fasta[chrom])

21531802

In [81]:
l = ['A','B','C','D','E']

In [83]:
l[[14] = ['N','N','N']

In [88]:
l[3:1:-1]

['N', 'N']

In [188]:
complement = {'A':'T','C':'G','G':'C','T':'A','N':'N'}


def delta_to_query_coord_fasta(fn,ref_fasta,query_fasta,query_chrom):
    """
    Read a nucmer delta file and.
    output the reference sequence in a string in 
    coordinates of the query sequence. Positions in the 
    query sequence that do not hava a reference alignment
    are marked 'N'
    """
    c = complement
    
    ix = 0
    ref_fasta_in_query_coord = ['N']*len(query_fasta[query_chrom])

    with open(fn,'r') as f:
        f.next()
        f.next()
        for line in f:
            if line[0] == ">":
                #print "starting alignment",line
                ref_chrom = line[1:].split()[0]
                ref_fa = ref_fasta[ref_chrom]
                new_chrom = True
                new_align = True
            elif new_align:
                #print "*******************"
                #print "new alignment",line
                start_ref = int(line.split()[0])
                start_query = int(line.split()[2])
                end_ref = int(line.split()[1])
                end_query = int(line.split()[3].strip())
                assert start_ref < end_ref,\
                        "referse strand in reference not implemented"
                pos_ref = start_ref - 1 
                if start_query < end_query:
                    pos_query = start_query -1
                    query_reverse = False
                    #print "*** forward alignment ***"
                else:
                    pos_query = start_query - 1
                    query_reverse = True
                    #print "*** reverse alignment ***"
                new_align = False
            elif line.strip() == '0':
                #alignment finished
                ref_str = ref_fa[pos_ref:end_ref-1]
                if not query_reverse:
                    ref_fasta_in_query_coord[pos_query:end_query-1] = ref_str
                else:
                    ref_str = "".join([complement[b] for b in ref_str])
                    try:
                        ref_fasta_in_query_coord[pos_query:end_query-1:-1] = ref_str
                    except:
                        print "end"
                        print ref_chrom, start_ref, end_ref, pos_ref, query_chrom, start_query, end_query , pos_query
                        raise
                 #debug
                #if not query_reverse:
                #    print "mac:"
                #    print "".join(ref_fasta_in_query_coord[start_query:end_query])
                #    print "ver:"
                #    print query_fasta[query_chrom][start_query:end_query]
                #else:
                #    print "mac:"
                #    print "".join(ref_fasta_in_query_coord[end_query:start_query])
                #    print "ver:"
                #    print query_fasta[query_chrom][end_query:start_query]

                    
                
                new_align = True
            else:
                #until next indel
                next_indel = abs(int(line.strip()))-1
 
                in_or_del = math.copysign(1,int(line.strip()))
                
                ref_str = ref_fa[pos_ref:pos_ref+next_indel]
                
                if not query_reverse:
                    ref_fasta_in_query_coord[pos_query:pos_query+next_indel] = ref_str
                else:
                    try:
                        ref_str = "".join([complement[b] for b in ref_str])
                        ref_fasta_in_query_coord[pos_query:pos_query-next_indel:-1] = ref_str
                    except:
                        print "within"
                        print ref_chrom, start_ref, end_ref, pos_ref, query_chrom, start_query, end_query , pos_query
                        raise
                
                #print 'pos_ref:', pos_ref
                #print ref_str
                #print 'pos_query', pos_query
                #if not query_reverse:
                #    print query_fasta[query_chrom][pos_query:pos_query+next_indel]
                #else:
                #    print query_fasta[query_chrom][pos_query:pos_query-next_indel:-1]
                #print "----------------------"
                pos_ref = pos_ref + next_indel + (1 if in_or_del > 0 else 0)
                pos_query = pos_query + (next_indel + (1 if in_or_del < 0 else 0))*(1-2*query_reverse)
            ix += 1
            #if ix > 2000:
            #    break
    return "".join(ref_fasta_in_query_coord)


In [56]:
map_fasta.wait(10)

In [ ]:
import Bio
from Bio import SeqIO

In [190]:
with open(eu("~/vervet_project/data/other_species/Macaca_mulatta/CR_1.0_in_vervet_coord_ref3500.fa".format(chrom)),'w') as oh:
    for chrom in chromosomes:
        delta_fn = jn(nucmer_dir,"nucmer_macaque_align_{}_filter_1_l200_u80.delta".format(chrom))
        ref_fasta_in_query_coord = delta_to_query_coord_fasta(delta_fn,mac_fasta,query_fasta=ver_fasta,query_chrom=chrom)
        seq = SeqIO.SeqRecord(Bio.Seq.Seq(ref_fasta_in_query_coord,Bio.Alphabet.DNAAlphabet),
                              id = chrom,
                              description = "")#mac CR_1.0 in C. sabaeus 1.1 coordinates
        SeqIO.write(seq, oh, "fasta")

In [189]:
ref_fasta_in_query_coord = delta_to_query_coord_fasta(delta_fn,mac_fasta,query_fasta=ver_fasta,query_chrom='CAE9')

In [147]:
SeqIO.SeqRecord?

In [146]:
SeqIO.write?

In [198]:
mac_in_ver_coord = pyfasta.Fasta(eu('~/vervet_project/data/other_species/Macaca_mulatta/CR_1.0_in_vervet_coord_ref3500.fa'))

In [203]:
mac_in_ver_coord['CAE1'][1000:1010]

u'GTGCAGTTAT'

In [14]:
with open(eu("~/vervet_project/data/other_species/Macaca_mulatta/mac_state_in_ver_coord.fasta"),'w') as fh:
    for chrom,r in itertools.izip(chromosomes[:-1],map_fasta.results):
        fh.write(">"+chrom+'\n')
        line_write(handle=fh,line_pos=0,string=r,line_len=80)


In [30]:
ver_fasta[chrom][196:215]

u'ATCTAGGGGCCCCAAGATG'

In [32]:
max(positions)

50887627

In [108]:
len('GGCTCACGCATGTAATCCCAGCACTTTGGGAGGCCGAGGCAGGCAGATCACAAGGTCAGGAGACCAAGACCATCCCGGCTAACATGGTGAAACCTCGTCTCTACTAC')

107

In [41]:
print len(mac_fasta_in_ver_coord)

50887628


In [42]:
print len(ver_fasta[chrom])

50890351


In [64]:
import numpy as np

0.30732416062879253

In [106]:
1732727-1732461

266

In [107]:
28745252 - 28744990

262

In [ ]:
Compare

In [4]:
ref_fn = eu("~/vervet_project/data/reference/reference_3500/"
            "3500_indID1_codeVRC_ref_sequencer3_seqType1_filtered0_version6.fasta")
ref_seq = pyfasta.Fasta(ref_fn)

In [3]:
mac_filtered = pyfasta.Fasta(eu('~/vervet_project/analyses/20150504_163_UnifiedGenotyper/_data/163_UG_ref3500_filtered_anc_state.fasta'))

In [29]:
mac_filtered_corrected = pyfasta.Fasta(eu('~/vervet_project/analyses/20150504_163_UnifiedGenotyper/_data/163_UG_ref3500_filtered_anc_state_corrected.fasta'))

In [23]:
from Bio import SeqIO

In [26]:
mac_rec = list(SeqIO.parse(eu("~/vervet_project/analyses/20150504_163_UnifiedGenotyper/_data/163_UG_ref3500_filtered_anc_state.fasta"),
                              "fasta"))

In [28]:
SeqIO.write(mac_rec, eu("~/vervet_project/analyses/20150504_163_UnifiedGenotyper/_data/163_UG_ref3500_filtered_anc_state_corrected.fasta"),
                               "fasta")

31

In [22]:
with open(eu("~/vervet_project/analyses/20150504_163_UnifiedGenotyper/_data/163_UG_ref3500_filtered_anc_state.fasta")) as ih,
     open(eu("~/vervet_project/analyses/20150504_163_UnifiedGenotyper/_data/163_UG_ref3500_filtered_anc_state.fasta"),'w') as oh:

        seq = SeqIO.SeqRecord(Bio.Seq.Seq(ref_fasta_in_query_coord,Bio.Alphabet.DNAAlphabet),
                              id = chrom,
                              description = "")#mac CR_1.0 in C. sabaeus 1.1 coordinates
        SeqIO.write(seq, oh, "fasta")

['__abstractmethods__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__doc__',
 '__eq__',
 '__format__',
 '__getattribute__',
 '__getitem__',
 '__hash__',
 '__init__',
 '__iter__',
 '__len__',
 '__metaclass__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_cache',
 '_abc_negative_cache',
 '_abc_negative_cache_version',
 '_abc_registry',
 '_seq_from_keys',
 'as_kmers',
 'chr',
 'fasta_name',
 'gen_seqs_with_headers',
 'get',
 'index',
 'items',
 'iteritems',
 'iterkeys',
 'itervalues',
 'key_fn',
 'keys',
 'prepared',
 'record_class',
 'sequence',
 'values']

In [ ]:
mac_filtered_corrected

In [30]:
for i,chrom in enumerate(chromosomes):
    print chrom, len(ref_seq[chrom])-len(mac_filtered_corrected[chrom])

CAE1 0
CAE2 0
CAE3 0
CAE4 0
CAE5 0
CAE6 0
CAE7 0
CAE8 0
CAE9 0
CAE10 0
CAE11 0
CAE12 0
CAE13 0
CAE14 0
CAE15 0
CAE16 0
CAE17 0
CAE18 0
CAE19 0
CAE20 0
CAE21 0
CAE22 0
CAE23 0
CAE24 0
CAE25 0
CAE26 0
CAE27 0
CAE28 0
CAE29 0
CAEX 0
CAEY 0


In [13]:
problems = []
with open(eu('~/vervet_project/analyses/20150504_163_UnifiedGenotyper/_data/163_UG_ref3500_filtered_anc_state.fasta')) as f:
    for i,line in enumerate(f):
        if len(line)!=81:
            problems.append((i,line))
        if len(problems)>100:
            break

In [10]:
pyfasta

80

In [14]:
len(problems)

74

In [18]:
2069029-1575451

493578

In [20]:
493578/70605.

6.990694710006373

In [17]:
for p in problems:
    print p[0], p[1]

0 >CAE1

1575450 NNNNNNNNNN

1575451 >CAE2

2069029 NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN

2069680 NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN

2705118 NNN

2705119 >CAE3

3856897 NNNNNNNNNNNNNNN

3856898 >CAE4

4420412 NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN

4425713 NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN

4994528 NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN

4994529 >CAE5

5937029 NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN

5937030 >CAE6

6207676 TCTCAAAANNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN

6215337 NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN

6573160 NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN

6573161 >CAE7

6784548 NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN

6785315 NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN